In [147]:
from yahoo_fin import stock_info as si
from datetime import date
from datetime import timedelta
import os
from twilio.rest import Client
from dotenv import load_dotenv


load_dotenv("../twilio.env")
account_sid = os.getenv("account_sid_twilio")
auth_token = os.getenv("auth_token_twilio")
sender = os.getenv("sender_twilio")
bindings = os.getenv("bindings_twilio")
client = Client(account_sid, auth_token)



In [148]:
today = date.today()

In [149]:
yesterday = today - timedelta(days = 6)

In [150]:
yesterday

datetime.date(2021, 5, 8)

In [151]:
today = date.today()
btc = get_data("btc-usd", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")
eth = get_data("eth-usd", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")
spy = get_data("spy", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")
gold = get_data("GLD", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")
tlt = get_data("tlt", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")
usd = get_data("dx-y.nyb", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")

In [152]:
column = ['BTC', 'ETH', 'SPY', 'TLT', 'GLD', 'USD']

combined_df = pd.concat([btc['close'], eth['close'], spy['close'], gold['close'], tlt['close'], usd['close']], axis="columns", join="inner")
combined_df.columns = column
combined_df


,BTC,ETH,SPY,TLT,GLD,USD
2021-05-10,56704.574219,4168.701172,417.940002,172.110001,137.820007,90.209999
2021-05-11,49150.535156,3785.848633,414.209991,172.139999,137.039993,90.139999
2021-05-12,49716.191406,3715.148438,405.410004,170.429993,135.600006,90.709999


In [153]:
combined_df.dropna()

,BTC,ETH,SPY,TLT,GLD,USD
2021-05-10,56704.574219,4168.701172,417.940002,172.110001,137.820007,90.209999
2021-05-11,49150.535156,3785.848633,414.209991,172.139999,137.039993,90.139999
2021-05-12,49716.191406,3715.148438,405.410004,170.429993,135.600006,90.709999


In [168]:
last = combined_df.iloc[-1]
today = combined_df.iloc[-2]

In [169]:
today

BTC    49150.535156
ETH     3785.848633
SPY      414.209991
TLT      172.139999
GLD      137.039993
USD       90.139999
Name: 2021-05-11 00:00:00, dtype: float64

In [170]:
last

BTC    49716.191406
ETH     3715.148438
SPY      405.410004
TLT      170.429993
GLD      135.600006
USD       90.709999
Name: 2021-05-12 00:00:00, dtype: float64

In [171]:
daily_change = ((today - last) / last) * 100
daily_change 

BTC   -1.137771
ETH    1.903025
SPY    2.170639
TLT    1.003348
GLD    1.061937
USD   -0.628376
dtype: float64

In [172]:
daily_change.iloc[0]

-1.1377706819450562

In [176]:
twilio_message1 = ''

for index, asset in daily_change.iteritems(): 
    if asset > 1:  
        twilio_message1 = twilio_message1 + " " + f"{index} - BUY" + "\n"
    elif asset < -1:
        twilio_message1 = twilio_message1 + " " + f"{index} - SELL" + "\n"
    else:
        twilio_message1 = twilio_message1 + " " + f"{index} - HOLD" + "\n"


twilio_message1

' BTC - SELL\n ETH - BUY\n SPY - BUY\n TLT - BUY\n GLD - BUY\n USD - HOLD\n'

In [177]:
print(twilio_message1)

 BTC - SELL
 ETH - BUY
 SPY - BUY
 TLT - BUY
 GLD - BUY
 USD - HOLD



In [178]:
body = twilio_message1


message = client.messages.create(
                     body=body,            
                     from_= sender,
                     to =bindings
)
print(message.sid)


SM89b69d9adf65409ba12299468c6625ce
